# MoCap example

In [4]:
import csv
import pandas as pd
def custom_header_reader(fileSource:str):
    fileReader = open(fileSource,'r')
    csvReader = csv.reader(fileReader,delimiter='\t')
    for row, text in enumerate(csvReader):
        if row == 2: numMarkers = int(text[-1])
        elif row == 10: columnNames = text[:-1]; break
    return numMarkers, columnNames, fileReader

def line_reader(fileReader):
    for line in csv.reader(fileReader,delimiter='\t'):
        yield line
    fileReader.close()

EXAMPLE_PATH = "data/examples_for_thesis/t_0061.tsv"
numMarkers, columnNames, readerBuffer = custom_header_reader(EXAMPLE_PATH)
data = pd.DataFrame(line_reader(readerBuffer),columns=columnNames).astype(dict(zip(columnNames,[int,float,str]+[float]*len(columnNames[3:]))))
data = data.loc[data['Time'] >= 31.6]#.iloc[1379:,:]
posTable = data.iloc[:,3:]
posTableX = posTable.iloc[:,::3]
posTableY = posTable.iloc[:,1::3]
posTableZ = posTable.iloc[:,2::3]
posTableX.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6937 entries, 3160 to 10096
Data columns (total 82 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   61A_HeadFront X        6937 non-null   float64
 1   61A_HeadL X            6937 non-null   float64
 2   61A_HeadR X            6937 non-null   float64
 3   61A_Chest X            6937 non-null   float64
 4   61A_WaistLFront X      6937 non-null   float64
 5   61A_WaistRFront X      6937 non-null   float64
 6   61A_LShoulderTop X     6937 non-null   float64
 7   61A_RShoulderTop X     6937 non-null   float64
 8   61A_SpineThoracic2 X   6937 non-null   float64
 9   61A_SpineThoracic12 X  6937 non-null   float64
 10  61A_WaistBack X        6937 non-null   float64
 11  61A_WaistL X           6937 non-null   float64
 12  61A_WaistR X           6937 non-null   float64
 13  61A_LThighFrontLow X   6937 non-null   float64
 14  61A_RThighFrontLow X   6937 non-null   float64
 15  61A_L

In [5]:
import numpy as np
jointNames = [col[:-2] for col in posTableX.columns]
adjacencyMatrix = np.zeros((len(jointNames), len(jointNames)), dtype=int)
links = [
    (0,1),(1,2),(0,2), # testa
    (7,36),(36,38),(38,40),(40,36),(7,38),(7,40),  # spalla dx
    (6,35),(6,37),(6,39),(37,39),(39,35),(35,37),   # spalla sx
    (40,34),(38,32),(34,30),(32,30),(34,32),    # braccio dx
    (37,31),(33,31),(33,39),(31,29),(33,29),    # braccio sx
    (4,11),(5,4),(12,5),(12,10),(11,10),    # bacino
    (10,9),(9,8),(12,7),(11,6),(3,4),(3,5),(3,6),(3,7),(1,8),(2,8),(8,6),(8,7), # busto
    (5,14),(4,13),(13,15),(13,16),(13,19),(14,18),(18,20),(14,17),(17,20),(16,19),(15,19),  # quadricipiti
    (17,24),(18,22),(16,21),(15,23),(24,22),(28,26),(26,22),    # caviglie parte 1
    (21,23),(23,27),(25,27),(25,21),(24,28),(20,24),(20,22),(19,23),(19,21),    # caviglie parte 2
    (10,16),(11,15),(12,17),(10,18) # piedi
]

for (fr,to) in links:
    adjacencyMatrix[fr,to] = 1
    adjacencyMatrix[to,fr] = 1
    adjacencyMatrix[fr+41,to+41] = 1
    adjacencyMatrix[to+41,fr+41] = 1

In [6]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

plt.close("all")
fig = plt.figure(figsize=(12,12))
sub_plot_position = [0, -0.2, 1, 1.5]
ax:plt.Axes = fig.add_subplot(sub_plot_position, projection='3d')
ax.set_box_aspect((2, 1, 0.6))
ax.view_init(elev=14, azim=-90)
ax.set_yticks(range(0,2000,500))
ax.set_zticks(range(0,1750,500))

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values)*0-2400,np.nanmin(posTableY.values)*0,np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values)*0+2700,np.nanmax(posTableY.values)*0+2000,np.nanmax(posTableZ.values)]

#minMax[0,:] = [np.nanmin(posTableX.iloc[:1,:].values),np.nanmin(posTableY.iloc[:1,:].values),np.nanmin(posTableZ.iloc[:1,:].values)]
#minMax[1,:] = [np.nanmax(posTableX.iloc[:1,:].values),np.nanmax(posTableY.iloc[:1,:].values),np.nanmax(posTableZ.iloc[:1,:].values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=5,c="r")

    #for i, col_name in enumerate(posTableX.columns):
    #    ax.text(filteredX[i], filteredY[i], filteredZ[i], str(i), horizontalalignment='center',fontsize=6, color='black')

    # Draw lines based on the adjacency matrix
    for i in range(adjacencyMatrix.shape[0]):
        for j in range(i + 1, adjacencyMatrix.shape[1]):
            if adjacencyMatrix[i, j] == 1:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]], color='black',linewidth=0.9)

    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])
    ax.set_yticks(range(0,2000,500))
    ax.set_zticks(range(0,1750,500))
    
    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg


# Cutting Videos

In [ ]:
try:
    import cv2
    import tdqm
except:
    %pip install opencv-python
    %pip install tdqm
    import cv2

In [23]:
from os.path import join, exists
from os import makedirs
from tqdm import tqdm
import cv2
import numpy as np
import matplotlib.pyplot as plt

jointsFrom = [1, 3, 5, 7, 2, 4, 6, 8, 8,10,11,13,15,17,12,14,16,18,18]
jointsTo =   [3, 5, 7, 8, 4, 6, 9, 9,10,18,13,15,17,18,14,16,19,19,20]
edges = np.array(list(zip(jointsFrom,jointsTo)))
edgeNames = ['Lfoot-Lankle',
          'Lankle-Lknee',           
          'Lknee-Lhip',
          'Lhip-Chip',
          'Rfoot-Rankle',           
          'Rankle-Rknee',           
          'Rknee-Rhip',
          'Rhip-Chip',
          'Chip-spine',
          'spine-Cshoulder',
          'Lhand-Lwrist',
          'Lwrist-Lelbow',
          'Lelbow-Lshoulder',
          'Lshoulder-Cshoulder',    
          'Rhand-Rwrist',
          'Rwrist-Relbow',
          'Relbow-Rshoulder',       
          'Rshoulder-Cshoulder',    
          'Cshoulder-head']    

PATH = '/media/gagg/SSD/OoM/MoCap/Wholodance/compressed_videos'     # local path of mocap videos
OUTPUT_ROOT_FOLDER = PATH+'/cut_videos'     # creates a child folder
VIDEOS_SUFFIX = '_video01.mp4'      # frontal view
PRE_POST_EXTRA_SECONDS = 1          # adds extra seconds to resulting video for better observe OoMs
OVERWRITE = False                   # set to true to overwrite cut videos if already present

with open('/'.join(PATH.split('/')[:-2])+'/annotations.txt','r') as file:
    annotations = file.read().replace(' ','').split('\n')[1:-1]

annotations = [ann for ann in annotations if 'ok' in ann]
must_rewind_video = True
for i,ann in tqdm(enumerate(annotations), total=len(annotations), desc="Processing videos",ncols=int(np.clip(len(annotations)*5,0,150))):
    ann = ann.split(',')[:6]

    folder,name_file,fragId,OoM_edges,start_time,end_time = ann[0],ann[1],int(ann[2]),[tuple(map(int, ann[3].strip('[]').split(';')[i].split('-'))) for i in range(0, int(';' in ann[3])+1) ],float(ann[4]),float(ann[5])
    must_open_new_video = i == 0 or fragId == 0 or start_time-PRE_POST_EXTRA_SECONDS <= float(annotations[i-1].split(',')[5])+PRE_POST_EXTRA_SECONDS
    must_close_video = i == len(annotations)-1 or int(annotations[i+1].split(',')[2])==0 or end_time+PRE_POST_EXTRA_SECONDS >= float(annotations[i+1].split(',')[4])-PRE_POST_EXTRA_SECONDS

    if must_open_new_video:
        frame_number = 0
        video_path = join(PATH,folder,folder+'_'+name_file+VIDEOS_SUFFIX)
        cap = cv2.VideoCapture(video_path)

    # Check if the video file was opened successfully
    if not cap.isOpened():
        print("Error opening video file")
        exit()

    # Get the frames per second (fps) of the video
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Calculate the frame indexes for start and end times
    start_frame = int(np.clip(start_time-PRE_POST_EXTRA_SECONDS,0.,np.inf) * fps)
    end_frame = int(np.clip(end_time+PRE_POST_EXTRA_SECONDS,0.,np.floor(cap.get(cv2.CAP_PROP_FRAME_COUNT)/fps)-1) * fps)

    # Read and save the frames within the specified range
    frames = []
    while True:
        _, frame = cap.read()
        if start_frame <= frame_number <= end_frame:
            cv2.putText(img=frame,
                        text=' / '.join([edgeNames[i] for i in np.where(np.all(edges[:,np.newaxis,:] == OoM_edges, axis=2))[0].tolist()]), 
                        org=(10,20), 
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                        fontScale=0.5, 
                        color=(255,255,255), 
                        thickness=1)
            frames.append(frame)
        elif frame_number > end_frame:
            break
        frame_number += 1
    if must_close_video:
        cap.release()

    # Save the frames as a new video
    output_folder = join(OUTPUT_ROOT_FOLDER,folder)

    if not exists(output_folder): makedirs(output_folder)
    
    output_path = join(output_folder,folder+'_'+name_file+'_frag'+str(fragId)+'.mp4')
    if not exists(output_path) or OVERWRITE:
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frames[0].shape[1], frames[0].shape[0]))
        for frame in frames:
            out.write(frame)
        out.release()

Processing videos:   0%|                                                                                                       | 0/42 [00:00<?, ?it/s]

Processing videos: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [02:03<00:00,  2.95s/it]
